In [26]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import plotly.graph_objects as go
import ipywidgets as widgets
from ipywidgets import interact

def parse_table(url, timelimit=3600):
    """
    parse a specific table to generate a dictionary with the runtimes and
    auxiliary information
    """
    resp = requests.get(url) 
    soup = BeautifulSoup(resp.text, features="html.parser")
    pre = soup.find_all('pre')
    
    stats = {}
    
    stats['date'] = pre[0].text.split('\n')[1].replace('=','').replace('-','').strip()
    stats['title'] = pre[0].text.split('\n')[2].strip()

    if url.find('lpsimp') >= 0:
        tab = pre[3].text.split('\n')
        _version = pre[2].text.split('\n')[1:-1]
        _version = [x.split()[0].rstrip(':') for x in _version]
        _score = tab[3].split()
        _solved = tab[4].split()[1:]
        solver = tab[6].split()[2:]
        stats['solver'] = solver
        nprobs = int(tab[6].split()[0])
        stats['nprobs'] = nprobs
        stats['score'] = {solver[i]:_score[i] for i in range(len(solver))}
        stats['solved'] = {solver[i]:_solved[i] for i in range(len(solver))}
        stats['version'] = {solver[i]:_version[i] for i in range(len(solver))}
        stats['timelimit'] = int(pre[3].text.split('\n')[-2].split()[1].replace(',',''))
        stats['times'] = pd.DataFrame([l.split() for l in tab[8:nprobs+8]], columns=['instance']+solver)

    else:
        tab = pre[2].text.split('\n')
        tabmark = [ind for ind,i in enumerate(tab) if i.startswith('=====')]
        _version = pre[1].text.split('\n')[1:-1]
        _version = [x.split()[0].rstrip(':') for x in _version]
        _score = tab[1].split()[1:]
        _solved = tab[2].split()[1:]
        solver = tab[4].split()[1:]
        stats['solver'] = solver
        nprobs = len(tab[tabmark[0]+3:tabmark[1]])
        stats['nprobs'] = nprobs
        stats['score'] = {solver[i]:_score[i] for i in range(len(solver))}
        stats['solved'] = {solver[i]:_solved[i] for i in range(len(solver))}
        if len(_version) == len(solver):
            stats['version'] = {solver[i]:_version[i] for i in range(len(solver))}
        else:
            stats['version'] = {solver[i]:solver[i] for i in range(len(solver))}
        stats['timelimit'] = timelimit
        stats['times'] = pd.DataFrame([l.split() for l in tab[tabmark[0]+3:tabmark[1]]], columns=['instance']+solver)
        
    return stats


def plot_benchmark(stats):
    """
    generate an interactive Plotly figure from the given dictionary
    """
    # clean up non-numeric values and replace with timelimit
    time = stats['times']
    for s in stats['solver']:
        time[s] = pd.to_numeric(stats['times'][s], errors='coerce')
    time.fillna(value=stats['timelimit'], inplace=True)
    fig = go.Figure()

    for s in stats['solver']:
        fig.add_trace(go.Scatter(x=time['instance'], y=time[s],
                        mode='markers',
                        name=stats['version'][s]))
    fig.update_layout(title=f'{stats["title"]} ({stats["date"]})', yaxis_type="log")
    fig.show()
    return fig

In [31]:
# %debug
# stats = parse_table('http://plato.asu.edu/ftp/lpsimp.html')
stats = parse_table('http://plato.asu.edu/ftp/qplib.html', 3600)
# stats = parse_table('http://plato.asu.edu/ftp/nonbinary.html', 10800)
# stats = parse_table('http://plato.asu.edu/ftp/cnconv.html', 10800)
# stats = parse_table('http://plato.asu.edu/ftp/convex.html', 7200)
# stats = parse_table('http://plato.asu.edu/ftp/cconvex.html', 3600)

In [38]:
urls = [('http://plato.asu.edu/ftp/lpsimp.html', 15000),
('http://plato.asu.edu/ftp/qplib.html', 3600),
('http://plato.asu.edu/ftp/nonbinary.html', 10800),
('http://plato.asu.edu/ftp/cnconv.html', 10800),
('http://plato.asu.edu/ftp/convex.html', 7200),
('http://plato.asu.edu/ftp/cconvex.html', 3600),
]
urls[0][0][:-5]

'http://plato.asu.edu/ftp/lpsimp'

In [32]:
time = stats['times']
for s in stats['solver']:
    time[s] = pd.to_numeric(stats['times'][s], errors='coerce')

    time.fillna(value=stats['timelimit'], inplace=True)

comp = widgets.Dropdown(value=stats['solver'][0], options=time.keys().drop('instance'), description='Base solver')

@interact(y=comp)
def plotit(y):
    fig = go.Figure()
    for s in time.keys().drop('instance'):
        if s == y:
            fig.add_trace(go.Scatter(x=time['instance'], y=time[y], mode='markers', name=stats['version'][s]))
        else:
            fig.add_trace(go.Bar(x=time['instance'], y=time[s]-time[y], name=stats['version'][s], base=0))
    
    fig.update_layout(title=f'Absolute time differences using {stats["version"][y]} as base solver ({stats["date"]})',
                      xaxis=dict(type='category')
                     )
    return fig

interactive(children=(Dropdown(description='Base solver', options=('BARON', 'SCIP', 'ANTIGONE', 'MINOTAUR', 'O…

In [ ]:
plots = ""

plots += f'<h2 id="simplex-lp">Simplex LP ({stats["date"]})</h2>\n'
plots += '<h3>Choose base solver for comparison:<h3>\n<ul>\n'

for s in time.keys().drop('instance'):
    fig = plotit(s)
#     fig.show()
    fig.write_html(f'docs/{s}.html', include_plotlyjs='cdn')
    plots += f'\t<li><a href={s}.html>{stats["version"][s]}</a></li>\n'
plots += '</ul>\n'

In [ ]:
top="""<!DOCTYPE html>
<html lang="en-US">
  <head>
    <meta charset="UTF-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1">

<!-- Begin Jekyll SEO tag v2.6.1 -->
<title>Visualizations of Mittelmann benchmarks | mittelmann-plots</title>
<meta name="generator" content="Jekyll v3.8.5" />
<meta property="og:title" content="Visualizations of Mittelmann benchmarks" />
<meta property="og:locale" content="en_US" />
<meta name="description" content="Visualizations of Mittelmann benchmarks" />
<meta property="og:description" content="Visualizations of Mittelmann benchmarks" />
<link rel="canonical" href="https://mattmilten.github.io/mittelmann-plots/" />
<meta property="og:url" content="https://mattmilten.github.io/mittelmann-plots/" />
<meta property="og:site_name" content="mittelmann-plots" />
<script type="application/ld+json">
{"@type":"WebSite","headline":"Visualizations of Mittelmann benchmarks","url":"https://github.com/mattmilten/mittelmann-plots","name":"mittelmann-plots","description":"Visualizations of Mittelmann benchmarks","@context":"https://schema.org"}</script>
<!-- End Jekyll SEO tag -->

    <link rel="stylesheet" href="/mittelmann-plots/assets/css/style.css?v=73b8bebba12f73a6e4f8873242ca033c15630844">
    <script src="https://cdn.plot.ly/plotly-latest.min.js"></script>
  </head>
  <body>
    <div class="container-lg px-3 my-5 markdown-body">
      <h1 id="visualizations-of-mittelmann-benchmarks">Visualizations of Mittelmann benchmarks</h1>

"""

bottom="""
      <p>older versions:
        <ul>
          <li><a href=lpcomp-2020-08-14.html>14 Aug 2020</a></li>
        </ul>
      </p>

      <div class="footer border-top border-gray-light mt-5 pt-3 text-right text-gray">
        This site is open source. <a href="https://github.com/mattmilten/mittelmann-plots">Improve this page</a>.
      </div>
      
    </div>
    <script src="https://cdnjs.cloudflare.com/ajax/libs/anchor-js/4.1.0/anchor.min.js" integrity="sha256-lZaRhKri35AyJSypXXs4o6OPFTbTmUoltBbDCbdzegg=" crossorigin="anonymous"></script>
    <script>anchors.add();</script>
    
  </body>
</html>
"""

In [ ]:
with open('docs/index.html', 'w') as index:
    index.write(top)
    index.write(plots)
    index.write(bottom)